# **Assignment 3**
* Yanna Polonia

## **Problem Statement**

### This data represents the condition of an electrical mini-substation based on readings from voltage, current and temperature sensor readings. A condition of ‘0’ represents a properly functioning device, while a condition of ‘1’ represents failure. The model is to be used to alert maintenance staff of when a device has failed so that they can repair the mini-substation as soon as possible.

## **Question 1:**

### **Which is worse for this use case, a false positive or a false negative?**

### <span style='color:blue'>**On a balanced situation being this mini-substation used to maintain critical activities going on and representing their repair cost (personnel to be deployed to fix it, parts and so on), a false negative will be worse because picturing the mini-substation is used for a hospital or an airport, its failure can end up in human damage.**</span>

### **What value of β would be suitable for an Fβ score?**

### <span style='color:blue'>**β should be higher than 1 to avoid false negative (so it could be 2 in this case), a high recall model is required**</span>

In [1]:
# Importing packages
from matplotlib import pyplot as plt
import numpy as np
import random
random.seed(0)
import turicreate as tc
import utils

## **Question 2:**

### **Loading the CSV file generated using SMLCode into an SFrame named data and displaying is content afterwards**

In [2]:
data = tc.SFrame.read_csv('398791_data.csv')
data

Finished parsing file /home/ypolonia/SupervisedML/Assignment3/398791_data.csv

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Parsing completed. Parsed 100 lines in 0.025964 secs.

Finished parsing file /home/ypolonia/SupervisedML/Assignment3/398791_data.csv

Parsing completed. Parsed 1000 lines in 0.026694 secs.

Condition,Voltage,Current,Temperature
1,66.41917850435287,410.76481898475686,62.64437488331776
1,66.01044424541274,412.0386278949937,31.590241857753487
1,66.72709296483444,411.6973882499786,63.931543113699966
1,66.74640459720754,413.5890653412429,54.1037712609902
1,67.03588659792423,411.3148941632079,35.49446439679747
1,64.96468623549002,413.86353032923665,52.50231143693807
1,65.01176068470157,409.9774492898164,35.4313846937502
1,65.35061040853004,411.3017302653269,55.49484739838955
1,65.91739768885215,412.7694787785817,59.77051701062758
1,66.11552269048278,412.4099489437384,64.67590391182634


### **Splitting the data into training/validation/testing sets using 80% / 10% / 10% respectively**
* Storing 80% of the data in the train dataset, the remain is divided 50/50 to store 10% of it for both: validation and test dataset

In [3]:
train, remain = data.random_split(.8, seed=0)

In [4]:
validation, test = remain.random_split(.5, seed=0)

In [5]:
# Verifying datasets proportions obtained
print('Datasets length obtained after splitting data:\n')
print(f"Training: {len(train)} \nValidation: {len(validation)} \nTest: {len(test)}")

Datasets length obtained after splitting data:

Training: 797 
Validation: 100 
Test: 103


## **Question 3:**

### **Is feature rescaling turned on by default for the function turicreate.logistic_classifier.create?**

### <span style='color:blue'>**Feature rescaling is by default set to "True" but this is an optional parameter in this function, the documentation states that: an l2-norm rescaling is performed to make sure that all features are of the same norm. Categorical features are also rescaled by rescaling the dummy variables that are used to represent them.  Previous explanation is focused on L2 as the function was designed to follow the ridge regression by deafult).**</span>

### **What scale (original or rescaled) are the coefficients given in?**

### <span style='color:blue'>**The coefficients are returned in original scale of the problem.**</span>

## **Question 4:**

### **Creating perceptrons using Turicreate to classify data with ‘Condition’ as the target. Experimenting with different values of hyperparameters to develop two different models**

In [11]:
value = 0.01    # Stores default value (reference), using 0.01 as it is the default one for L2 for the logistic_classifier
grid_search = []   # To store different values for L1 and L2 to create models based on them

for num in range (0 , 60, 10):
    if num * value == 0:         # This if allows two pair of values of L1 and L2, being one of them 0.01 and the other 0 and viceversa.
        grid_search.append({'L1': num * value, 'L2': value, 'feature_rescaling': random.randint(0, 1)})  
        grid_search.append({'L1': value, 'L2': num * value, 'feature_rescaling': random.randint(0, 1)})
    elif num * value != 0:      # Setting same value for L1 and L2 (values used are less than 1 - keeping 0.01 as reference value to get new values) 
        grid_search.append({'L1': num * value, 'L2': num * value, 'feature_rescaling': random.randint(0, 1)})
        
#tc.evaluation.confusion_matrix(

In [12]:
f_beta_scores = []       # To store f beta score of training predictions obtained
perceptrons = []         # To store each perceptron to create using the logistic classifier and training dataset 
precision_values = []    # To store precision values based on training predictions obtained and training dataset 
train_predictions = []   # To store training predictions based on each perceptron using the training dataset
validation_predictions = [] # To store training predictions based on each perceptron using the training dataset
train_confusion_matrix = [] # To store the training confusion matrix

# Looping to create a perceptron of each combination of L1/L2 generated before, using fixed values for parameters included below except for the mentioned ones.
for i in range(0, len(grid_search)):
    perceptrons.append(
        tc.logistic_classifier.create(  # 
        train, 
        target = 'Condition',
        features = ['Voltage', 'Current', 'Temperature'], 
        l2_penalty = grid_search[i]['L2'],
        l1_penalty = grid_search[i]['L1'],
        feature_rescaling = bool(grid_search[i]['feature_rescaling']),  
        max_iterations = 50,    # Set to 50 to see how many iterations of this amount each model needs to get better
        validation_set = validation,))
    
    train_predictions.append(perceptrons[i].predict(train))    # Adding training predictions generated to its list
    validation_predictions.append(perceptrons[i].predict(validation))    # Adding validation predictions generated to its list

    precision_values.append(tc.evaluation.precision(train['Condition'], train_predictions[i]))  # Adding precision values obtained to its list
    f_beta_scores.append(tc.evaluation.fbeta_score(train['Condition'], train_predictions[i], beta = 2.0)) # Adding f beta score values obtained to its list
    train_confusion_matrix.append(tc.evaluation.confusion_matrix(train['Condition'], train_predictions[i]))

Logistic regression:

--------------------------------------------------------

Number of examples          : 797

Number of classes           : 2

Number of feature columns   : 3

...

+-----------+----------+-----------+--------------+-------------------+---------------------+

| 0         | 1        | 1.000000  | 0.001286     | 0.498118          | 0.510000            |

| 1         | 2        | 0.000000  | 0.033953     | 0.501882          | 0.490000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

## **Question 5:**

### **Displaying required values of each created model**

In [13]:
# Using logistic classifier object: training / validation accuracy, training recall / time /  log loss / solver status and validation confusion matrix (values) to display below
for i in range(0, len(grid_search)):
    print(f'Model {i + 1} \n\nHyperparameters: {grid_search[i]}') 
    print(f'\nTraining Accuracy: {perceptrons[i].training_accuracy} \nValidation Accuracy: {perceptrons[i].validation_accuracy}')
    print(f'Recall: {perceptrons[i].training_recall} \nPrecision: {precision_values[i]} \nFβ score : {f_beta_scores[i]} \n{perceptrons[i].progress}') 
    print(f'Training Time: {perceptrons[i].training_time} \nTraining Log Loss: {perceptrons[i].training_log_loss} \nModel Status: {perceptrons[i].training_solver_status}')
    print(f'\nValidation Set Confusion Matrix:\n{perceptrons[i].validation_confusion_matrix} \nTraining Confusion Matrix:\n{train_confusion_matrix[i]}') 

Model 1 

Hyperparameters: {'L1': 0.0, 'L2': 0.01, 'feature_rescaling': 1}

Training Accuracy: 0.9121706398996235 
Validation Accuracy: 0.83
Recall: 0.9093198992443325 
Precision: 0.9139240506329114 
Fβ score : 0.9102370146243067 
+-----------+--------+--------------+-------------------+---------------------+
| Iteration | Passes | Elapsed Time | Training Accuracy | Validation Accuracy |
+-----------+--------+--------------+-------------------+---------------------+
|     1     |   2    |   0.001411   |      0.912171     |       0.830000      |
|     2     |   3    |   0.003886   |      0.912171     |       0.830000      |
|     3     |   4    |   0.006229   |      0.912171     |       0.830000      |
|     4     |   5    |   0.008567   |      0.912171     |       0.830000      |
+-----------+--------+--------------+-------------------+---------------------+
[4 rows x 5 columns]

Training Time: 0.009823 
Training Log Loss: 0.29667578372782805 
Model Status: SUCCESS: Optimal solution fo

## **Question 6:**

### <span style='color:blue'>**Model 1 is the most optimal model considering it took less iterations (4) compared to the rest of models evaluated with feature_rescaling = True (1) to find an optimal solution. The rest of values displayed : precision, recall, Fβ score (2.0), training / validation accuracy have the same values for all the models. Besides, Model 1 exhibits the lower value of training log loss (this last one is an indicative of how close the prediction probability is to the corresponding actual/true value (0 or 1 in case of binary classification), penalizing inaccurate predictions with higher values. Lower log-loss indicates better model performance,being the most important classification metric based on probabilities).</span>**

**Notes**: 
>**Models with feature_rescaling = False (0) were discarded because the recall and f beta score (2.0) is equal to 0 and the precision is None (undetermine value), having just true negative and false negative values in their confusion matrix.**

>**For feature_rescaling hyperparameter we have a random value assigned so this answer may not match the results each time the cells are run**

## **Question 7:**

In [9]:
test_predictions = perceptrons[0].predict(test)

In [10]:
print('Model 1:')
print('\nTest Predictions: ',perceptrons[0].predict(test))
print('\nConfusion Matrix:\n', tc.evaluation.confusion_matrix(test['Condition'], test_predictions))
print('Test accuracy: ', tc.evaluation.accuracy(test['Condition'], test_predictions))
print('Precision:', tc.evaluation.precision(test['Condition'], test_predictions))
print('Recall:', tc.evaluation.recall(test['Condition'], test_predictions))
print('F-beta score (2.0):', tc.evaluation.fbeta_score(test['Condition'], test_predictions, beta = 2))

Model 1:

Test Predictions:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... ]

Confusion Matrix:
 +--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      1       |        1        |   47  |
|      0       |        1        |   4   |
|      0       |        0        |   47  |
|      1       |        0        |   5   |
+--------------+-----------------+-------+
[4 rows x 3 columns]

Test accuracy:  0.912621359223301
Precision: 0.9215686274509803
Recall: 0.9038461538461539
F-beta score (2.0): 0.9073359073359073
